## Практическое задание к уроку 6 по теме "Классификация текста. Анализ тональности текста".

Взять ноутбук colab_text_classification_part1.ipynb который разбирали на занятии и
добавить пункты которые мы пропустили  
1. Посмотрите на токены если будут мусорные добавьте их в стоп слова и обучите
заново  
2. Проверьте изменилось ли качество при лемматизации/и без неё  
3. Замените все токены которые принадлежат сущностям на их тег. Проверьте
изменилось ли качество после этого 

Загрузим библиотеки и датасеты:

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
import spacy
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../Теория/Lesson_6/train.tsv", delimiter="\t")
test_df = pd.read_csv("../../Теория/Lesson_6/test.tsv", delimiter="\t")

1. Минимальная обработка.

Сделаем минимальную предобработку, как на уроке -  
удалим символы пропуска строки:

In [3]:
train_df['review'] = train_df['review'].apply(lambda x: x.replace('<br />', ' '))
test_df['review'] = test_df['review'].apply(lambda x: x.replace('<br />', ' '))

Обучим модель на текущем состоянии датасета. В качестве  
векторайзера возьмём CountVectorizer, т.к. он показал себя  
лучше, чем Tfidf. Возьмём униграммы и биграммы слов и не  
забудем отмасштабировать данные перед логистической регрессией:

In [4]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
scaler = MaxAbsScaler()
classifier = LogisticRegression()

model = make_pipeline(vectorizer, scaler, classifier)
model.fit(train_df['review'], train_df['is_positive']);

Напишем функцию для оценки моделей, т.к. этот кусок кода  
будем часто использовать:

In [5]:
def eval_model(model):
    pred = model.predict(test_df['review'])
    return accuracy_score(test_df['is_positive'], pred)

Будем записывать все результаты в словарь для последующей  
визуализации:

In [6]:
results = {}

In [7]:
results['Minimum preprocessing'] = eval_model(model)
print(results['Minimum preprocessing'])

0.90376


2. "Мусорные" токены.

Для обработки "лишних" токенов будем использовать аргумент  
векторайзера `max_df`, который будет фильтровать наиболее  
часто встречающиеся токены:

In [8]:
vectorizer = CountVectorizer(ngram_range=(1, 2), max_df=0.4)
model = make_pipeline(vectorizer, scaler, classifier)
model.fit(train_df['review'], train_df['is_positive']);

In [9]:
results['Min + exclude stopwords'] = eval_model(model)
print(results['Min + exclude stopwords'])

0.90544


Результат стал немного лучше.

3. Лемматизация.

Для лемматизации и замены токенов на сущности в следующем  
задании воспользуемся библиотекой spacy. Парсер использовать  
не будем, т.к. текст у нас уже разбит на объекты.

In [10]:
nlp = spacy.load('en_core_web_sm', disable=['parser'])

In [11]:
docs = nlp.pipe(train_df['review'])

In [12]:
lemmatized = []
for doc in tqdm(docs, total=len(train_df), position=0):
     lemmatized.append(' '.join(token.lemma_ for token in doc))

100%|█████████████████████████████████████| 25000/25000 [05:32<00:00, 75.18it/s]


In [13]:
train_df['review_lemma'] = lemmatized

In [14]:
model.fit(train_df['review_lemma'], train_df['is_positive'])
results['Min + lemmatization'] = eval_model(model)
print(results['Min + lemmatization'])

0.87056


Результат получился значительно хуже. Дополнительно проверим  
ещё результат работы, где векторайзер не фильтрует слова по  
частоте. Вполне возможно, что после лемматизации часть важных  
слов отфильтровалась, т.к. слова объединились и превысили порог.

In [15]:
vectorizer.max_df = 1.0

In [16]:
model.fit(train_df['review_lemma'], train_df['is_positive'])
results['Min + stopwords + lemmatization'] = eval_model(model)
print(results['Min + stopwords + lemmatization'])

0.86772


Нет, без фильтрации токенов результат получился ещё хуже.

4. Замена токенов на сущности.

In [17]:
docs = nlp.pipe(train_df['review'])

In [18]:
replaced = []
for doc in tqdm(docs, total=len(train_df), position=0):
    spam = []
    for token in doc:
        spam.append(token.ent_type_ if token.ent_type else token.text)
    replaced.append(' '.join(spam))

100%|█████████████████████████████████████| 25000/25000 [05:31<00:00, 75.41it/s]


In [19]:
train_df['review_replace'] = replaced

Здесь тоже проверим оба варианта: с фильтрацией  
слов и без неё. Потому что сущности могут оказаться  
слишком частыми токенами и будут отфильтрованы.

In [20]:
model.fit(train_df['review_replace'], train_df['is_positive'])
results['Min + replace entities'] = eval_model(model)
print(results['Min + replace entities'])

0.9022


In [21]:
vectorizer.max_df = 0.4

In [22]:
model.fit(train_df['review_replace'], train_df['is_positive'])
results['Min + stopwords + entities'] = eval_model(model)
print(results['Min + stopwords + entities'])

0.90252


Сведём результаты в таблицу:

In [23]:
pd.DataFrame(results.values(), index=results.keys(), 
             columns=['accuracy']).sort_values('accuracy', ascending=False)

,accuracy
Min + exclude stopwords,0.90544
Minimum preprocessing,0.90376
Min + stopwords + entities,0.90252
Min + replace entities,0.90220
Min + lemmatization,0.87056
Min + stopwords + lemmatization,0.86772


Лучший результат показала минимальная предобработка  
с исключением часто встречающихся токенов. Лемматизация  
оказала отрицательный эффект на метрику. Замена токенов  
на тэги сущностей чуть ухудшила результат.